import required packages and setup environment

In [1]:
import arcpy, os, re
from arcpy.sa import *
import pandas as pd
import numpy as np
import itertools as it
from IPython.display import clear_output

wd = os.getcwd()
arcpy.env.workspace = os.path.join(wd, "DataExploration.gdb")
scratch = os.path.join(wd, "scratch")
outputs = os.path.join(wd, "outputs")

arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("NAD 1983 BC Environment Albers")
arcpy.env.overwriteOutput = True
arcpy.env.snapRaster = r"H:\Muise\forestMask\ForMask_UTM-7S-11S_1984.tif"
arcpy.env.cellSize = 30
arcpy.CheckOutExtension("spatial")

'CheckedOut'

generate masked singlepart BEC zones to create GPE with

In [74]:
BEC = "Terr_BEC_Subzones"
mask = "Terr_ParksConsComplexes"

eraseOutput = "Terr_BEC_Erase"
mtsOutput = "Terr_BEC_Subzones_Multipart"

Testing subzone buffer

In [75]:
#erasedBEC = arcpy.Erase_analysis(BEC, mask, eraseOutput)
#arcpy.MultipartToSinglepart_management(erasedBEC, mtsOutput)

create two features classes: one for the non-complex protected areas, and one for the dissolved complexes

In [83]:
parkLayer = "Terr_ParksConsComplexes"

selection = 'Complex IS NOT NULL'

selected = arcpy.SelectLayerByAttribute_management(parkLayer, "NEW_SELECTION", selection)
complexes = arcpy.CopyFeatures_management(selected, os.path.join(scratch, "onlyComplexes.shp"))

dissolveComplexes = arcpy.Dissolve_management(complexes, "onlyComplexes",
                          "Complex", "", "MULTI_PART", 
                          "DISSOLVE_LINES")

arcpy.Delete_management(complexes)

selection2 = 'Complex IS NULL'
selected2 = arcpy.SelectLayerByAttribute_management(parkLayer, "NEW_SELECTION", selection2)
parks = arcpy.CopyFeatures_management(selected2, "onlyParks")
print("done")

done


In [81]:
parkLayer = "Terr_ParksConsComplexes"

selection = 'Complex IS NOT NULL'

#select_analysis should be faster than select then copy
complexes = arcpy.Select_analysis(parkLayer, os.path.join(scratch, "onlyComplexes.shp"), selection)

#selected = arcpy.SelectLayerByAttribute_management(parkLayer, "NEW_SELECTION", selection)
#complexes = arcpy.CopyFeatures_management(selected, os.path.join(scratch, "onlyComplexes.shp"))

dissolveComplexes = arcpy.Dissolve_management(complexes, "onlyComplexes",
                          "Complex", "", "MULTI_PART", 
                          "DISSOLVE_LINES")

arcpy.Delete_management(complexes)

selection2 = 'Complex IS NULL'

#select_analysis should be faster than select then copy
parks = arcpy.Select_analysis(parkLayer, os.path.join(scratch, "onlyParks"), selection2)

#selected2 = arcpy.SelectLayerByAttribute_management(parkLayer, "NEW_SELECTION", selection2)
#parks = arcpy.CopyFeatures_management(selected2, "onlyParks")
print("done")

done


In [84]:
regex = re.compile('[^a-zA-Z]')

with arcpy.da.UpdateCursor(parks, "PROTECTED_LANDS_NAME") as cursor:
    #removes all non-alphabetic characters for each polygon in the PROTECTED_LANDS_NAME field
    for row in cursor:
        row[0] = regex.sub('', row[0].title())

        # Update the cursor with the updated list
        cursor.updateRow(row)

generates the BEC GPE for a park, the select by attribute needs to be changed to updateCursor

In [92]:
def becGpeCreation(parkLayer, park, field, outList):
    #makes it simpler to do a select by attribute
    field = arcpy.AddFieldDelimiters(parkLayer, field)
    selection = "{field} = '{val}'".format(field=field, val=park)

    #removes all non alphabetic characters from the park name
    regex = re.compile('[^a-zA-Z]')
    gdbName = regex.sub('', park)

    #creates a folder and saves a file name
    #if you fuck up again and delete everything, change outputs in this line to gdbLocation and uncomment it
    #in the initialization cell
    newGdb = os.path.join(outputs, gdbName)
    if not os.path.isdir(newGdb):
        os.makedirs(os.path.join(newGdb))
    parkSaveName = os.path.join(str(newGdb), gdbName + ".shp")

    outList.append(parkSaveName)

    #selected parameterized park, saves it to the new geodatabase
    #select_analysis faster than select layer and copy features
    parkFeature = arcpy.Select_analysis(parkLayer, parkSaveName, selection)
    
    
    #selected = arcpy.SelectLayerByAttribute_management(parkLayer, "NEW_SELECTION", selection)
    #parkFeature = arcpy.CopyFeatures_management(selected, parkSaveName)

    #generates the GPE, saves it to the new geodatabase
    selectBEC = arcpy.SelectLayerByLocation_management(mtsOutput, 'intersect', parkFeature)
    becSaveName = os.path.join(str(newGdb), gdbName + "_BEC.shp")
    arcpy.CopyFeatures_management(selectBEC, becSaveName)
    
    becList.append(becSaveName)

In [86]:
def largest_contiguous(shapefile, save_location):

    
    print("converting to singlepart")
    singlepart_save = r"scratch\gpeSinglepart.shp" 

    singlePart = arcpy.MultipartToSinglepart_management(shapefile, singlepart_save) #make singlepart

    arr = arcpy.da.TableToNumPyArray(singlePart, 'SHAPE@AREA') #determine number of parts
    
    if len(arr) > 1: 
        #if there are more than 1 part, select only those that are larger than 40% of the largest part
        print("multiple polygons, adding field")
        field_names = [f.name for f in arcpy.ListFields(shapefile)]
        
        if "Shape_Area" not in field_names:
            arcpy.AddField_management(singlePart, "Shape_Area", "FLOAT")
            
        print("calculating geometry")
        arcpy.CalculateGeometryAttributes_management(singlePart, 
                                                 [["Shape_Area", "AREA"]], 
                                                 area_unit = "SQUARE_METERS")
        
        arr = arcpy.da.TableToNumPyArray(singlePart, 'Shape_Area')
        
        max_value = np.nanmax(arr.astype('float'))  
        

        where_clause = """"{}" {} {}""".format("Shape_Area", ">", str(int(max_value * .40)))
        print(where_clause)
        
        print("selecting and saving")
        arcpy.Select_analysis(singlePart, save_location, where_clause = where_clause)
    else:
        print("single polygon, saving")
        arcpy.CopyFeatures_management(singlePart, save_location)

In [87]:
def bufferProp(inShape, proportion = 1, sensitivity = 0.05, adjustment_rate = 0.05):
    
    print("math and initialization")

    arr = arcpy.da.TableToNumPyArray(inShape, 'SHAPE@AREA') #convert column to an array so i can pull the number
    geomArea = np.nanmax(arr.astype('float')) #total area
    r1 = math.sqrt(geomArea / math.pi) #radius of circle with total area
    r2 = math.sqrt(geomArea * 2 / math.pi) #radius of circle with double area
    buffer_length = r2 - r1 #difference of circle radius(to be buffered by)

    bec_location = inShape[:-4] + "_BEC" + inShape[-4:] #location of the BEC zones connected: for clipping by
    save_location = inShape[:-4] + "_GPE" + inShape[-4:] #final save location for when it works
    

    lowerBound = proportion - sensitivity #acceptable lower bound of the ratio of buffered area to park area
    upperBound = proportion + sensitivity #acceptable upper bound of the ratio of buffered area to park area

    ratio = None
    print("buffering")
    while (ratio is None or ratio < lowerBound or ratio > upperBound): #if ratio is not in bounds
        print(".", end = "") #sort of a progress bar
        if ratio == None: #start off with no ratio because the buffer doesnt exist yet
            ratio = ratio
            
        elif ratio > proportion:
            buffer_length *= (1 - adjustment_rate) #if ratio too small, decrease by adjustment rate

        elif ratio < proportion:
            buffer_length *= (1 + adjustment_rate) #if ratio too small, increase by adjustment rate
        
        #buffer by the buffer length and save to scratch folder
        buffered = arcpy.Buffer_analysis(inShape, os.path.join(scratch, "interBuffer.shp"), buffer_length)

        #if a ratio exists (has been buffered), then clip it to the nearby BEC zones
        if ratio != None:
            
            buffered = arcpy.Clip_analysis(buffered, bec_location, r"scratch\BufferClip")

        buff_arr = arcpy.da.TableToNumPyArray(buffered, 'SHAPE@AREA') #convert column to an array so i can pull the number
        if len(buff_arr) == 0 :
            break
        buffered_geomArea = np.nanmax(buff_arr.astype('float')) #total area of buffered shapefile

        previous_ratio = ratio


        ratio = buffered_geomArea / geomArea #calculate the ratio

        if previous_ratio is not None and (round(previous_ratio, 6) == round(ratio, 6)):
            break #if the ratio is the same between iterations, break the function since it is an island

        if lowerBound < ratio < upperBound: 
            #if the ratio is within bounds, run largest contiguous function

            print("largest contiguous")
            
            largest_contiguous(buffered, save_location) #get the largest contiguous features from the GPE

            return inShape, save_location

code to generate unique values in selected field: a workaround instead of using updateCursor for my specific use case

In [88]:
def unique_values(table , field):
    with arcpy.da.SearchCursor(table, [field]) as cursor:
        return ({row[0] for row in cursor})

In [89]:
def subzones(ppa, gpe, overwrite = True, min_size = 1000):
    
    print("init")

    expression = "Shape_Area < " + str(int(min_size))

    #generates save names for intersect
    folder = ppa.rsplit("\\", 1)[0]
    ppaName = ppa.rsplit('\\', 1)[-1].split(".")[0]
    gpeName = gpe.rsplit('\\', 1)[-1].split(".")[0]
    ppaSave = os.path.join(folder, ppaName + "_Subzones.shp")
    gpeSave = os.path.join(folder, gpeName + "_Subzones.shp")
    
    paceSaveSub = os.path.join(folder, ppaName + "_PACE_Subzones.shp")
    
    if overwrite is True or not os.path.exists(paceSaveSub):
        print("generating subzones")

        arcpy.CalculateField_management(ppa, "ppa_gpe", "'PPA'", "PYTHON3", field_type = "TEXT")
        arcpy.CalculateField_management(gpe, "ppa_gpe", "'GPE'", "PYTHON3", field_type = "TEXT")
        
        print("generating merged, unsubzoned")
        paceSave = os.path.join(folder, ppaName + "_PACE.shp")
        pace = arcpy.Merge_management([ppa, gpe], paceSave)
        
        print("intersecting for subzoned ppa")
        intersectedPpa = arcpy.Intersect_analysis([ppa, "Terr_BEC_Subzones"], r"scratch\intersectedPpa.shp")

        print("making singlepart")
        singlePart = arcpy.MultipartToSinglepart_management(intersectedPpa,
                                       r"scratch\ppaSinglepart.shp")
        
        #pull numbers, check if any below 1000
        ppa_arr = arcpy.da.TableToNumPyArray(singlePart, 'SHAPE@AREA')
        
        print(str(sum([i[0] < min_size for i in ppa_arr])), "slivers in PPA") #determines number of slivers

        if sum([i[0] < min_size for i in ppa_arr]) > 0:

            print("calculating geometry")
            arcpy.CalculateGeometryAttributes_management(singlePart, 
                                                         [["Shape_Area", "AREA"]], 
                                                         area_unit = "SQUARE_METERS")

            print("select for slivers")
            selection = arcpy.SelectLayerByAttribute_management(singlePart, 
                                                    "NEW_SELECTION", 
                                                    expression)        
            print("eliminate slivers")
            intersectedPpa = arcpy.Eliminate_management(selection, r"scratch/ppaEliminated")
        
        
        
        
        print("intersecting for subzoned gpe")
        intersectedGpe = arcpy.Intersect_analysis([gpe, "Terr_BEC_Subzones"], r"scratch\intersectedGpe.shp")

        print("making singlepart")
        singlePart = arcpy.MultipartToSinglepart_management(intersectedGpe,
                                       r"scratch\gpeSinglepart.shp")
        
        gpe_arr = arcpy.da.TableToNumPyArray(singlePart, 'SHAPE@AREA')
        
        print(str(sum([i[0] < min_size for i in gpe_arr])), "slivers in GPE") #determines number of slivers
        
        if sum([i[0] < min_size for i in gpe_arr]) > 0:
        
            print("calculating geometry")
            arcpy.CalculateGeometryAttributes_management(singlePart, 
                                                         [["Shape_Area", "AREA"]], 
                                                         area_unit = "SQUARE_METERS")

            print("select for slivers")
            selection = arcpy.SelectLayerByAttribute_management(singlePart, 
                                                    "NEW_SELECTION", 
                                                    expression)        
            print("eliminate slivers")
            intersectedGpe = arcpy.Eliminate_management(selection, r"scratch/gpeEliminated")
        
        print("merge eliminated shapefiles")
        pace = arcpy.Merge_management([intersectedPpa, intersectedGpe], paceSaveSub)
        
        print("add myFID field for data aggregation")
        arcpy.CalculateField_management(paceSaveSub, "myFID", "!FID!", "PYTHON3", field_type = "SHORT")

    return(paceSaveSub)
        

In [90]:
parkNames = unique_values(parks, "PROTECTED_LANDS_NAME")
complexNames = unique_values(dissolveComplexes, "Complex")
#parkNames = ("JoffreLakesPark", "MountSeymourPark", "StuartRiverParkLowerSite")
#complexNames = ("Gar", "Spats", "Strath")

In [25]:
#initialize lists to merge at end
parkList = []
becList = []

#becGpeCreation(parks, "EkwanLakeProtectedArea", "PROTECTED_LANDS_NAME")
num_files = len(parkNames)
num_done = 0
for park in parkNames:
    
    clear_output(True)
    num_done += 1
    print(str(num_done), "/", str(num_files))
    print(park)
    
    becGpeCreation(parks, park, "PROTECTED_LANDS_NAME", parkList)
    
print("done iteration, now merging")

done iteration, now merging
done merging, running GPE buffer 

first attempt: no adjustment
None
2.5754162072830487
1.4361960323791554
1.3634911063172632
1.2945516318019825
1.2291336969938422
1.1669145554451352
1.107664132420982
1.0514241077126847
final ratio: 0.9980228469213434
number of iterations: 9
first attempt: no adjustment
None
2.1989005219031577
1.1308643636810578
1.0669846475876439
final ratio: 1.0070168494594796
number of iterations: 4
first attempt: no adjustment
None
3.776740411413477
2.166053385516996
2.0776107459915223
1.9909292266467764
1.9066093827563475
1.824545317901134
1.7440431601128688
1.6668777327656399
1.5933295571774548
1.5226533280335912
1.4557262063398428
1.391970305441338
1.3310941233709428
1.272650891025987
1.2166637963633078
1.1630715784664454
1.1111566054105346
1.0608450020849898
final ratio: 1.0126761199474636
number of iterations: 19
first attempt: no adjustment
None
4.4076332463435355
1.5258014635382413
1.4472960565639235
1.3730906920355759
1.302909812

KeyboardInterrupt: 

In [40]:
#find proper shapefiles in case of a crash
shapefiles = []
for root, dirs, files in os.walk("outputs"):
    for file in files:
        if file.endswith(".shp") and not (file.endswith("GPE.shp") 
                                          or file.endswith("BEC.shp") 
                                          or file.endswith("PACE.shp")
                                          or file.endswith("Subzones.shp")):
             shapefiles.append(os.path.join(root, file))
parkList = shapefiles
del shapefiles

In [56]:
buffered_parks = []
parkGPEs = []
parks_subzoned = []
num_done = 0

In [62]:
num_files = len(parkList)

for shapefile in parkList[num_done:]:
    clear_output(True)
    
    #if you need to resume from a crash, change num_done in the previous cell to the print statement below - 1
    print(str(num_done + 1), "/", str(num_files))
    print(shapefile)
    print("gpe creation")
    
    buffer_output = bufferProp(shapefile)
    if buffer_output is not None:
        buffered_parks.append(buffer_output[0])
        parkGPEs.append(buffer_output[1])
        clear_output(True)
    
        
        print(str(num_done + 1), "/", str(num_files))
        print(shapefile)
        print("subzones")
        parks_subzoned.append(subzones(buffer_output[0], buffer_output[1]))
    num_done += 1
print("done w/ gpes and subzoning")

954 / 954
outputs\ZumtelaBayConservancy\ZumtelaBayConservancy.shp
subzones
init
generating subzones
generating merged, unsubzoned
intersecting for subzoned ppa
making singlepart
0 slivers in PPA
intersecting for subzoned gpe
making singlepart
0 slivers in GPE
merge eliminated shapefiles
add myFID field for data aggregation
done w/ gpes and subzoning


shapefiles = []
for root, dirs, files in os.walk("outputs"):
    for file in files:
        if file.endswith("GPE.shp"):
             shapefiles.append(os.path.join(root, file))
parkGPEs = shapefiles
buffered_parks = [x[:-8] + x[-4:] for x in shapefiles]
del shapefiles

In [93]:
#initialize lists to merge at end
complexesList = []
becList = []
num_files = len(complexNames)
num_done = 0

for Complex in complexNames:
    num_done += 1
    clear_output(True)
    print(str(num_done), "/", str(num_files))
    print(Complex)
    
    becGpeCreation(dissolveComplexes, Complex, "Complex", complexesList)

25 / 25
Hakai


In [102]:
buffered_complexes = []
complexesGPEs = []
complexes_subzoned = []
num_done = 0

In [104]:
num_files = len(complexesList)

for shapefile in complexesList[num_done:]:
    
    clear_output(True)
    
    
    print(str(num_done + 1), "/", str(num_files))
    print(shapefile)
    
    buffer_output = bufferProp(shapefile)

    if buffer_output is not None:
        clear_output(True)
        print(str(num_done + 1), "/", str(num_files))
        print(shapefile)
        print("subzones")
        
        
        buffered_complexes.append(buffer_output[0])
        complexesGPEs.append(buffer_output[1])
        complexes_subzoned.append(subzones(buffer_output[0], buffer_output[1]))
    num_done += 1
print("done w/ gpes and subzoning")

25 / 25
C:\Users\evanmuis.stu\Sync\Masters\Data\outputs\Hakai\Hakai.shp
math and initialization
buffering
...........done w/ gpes and subzoning


myDict = {testParks: ("PPA", "Park"), 
          parkGPEs: ("GPE", "Park"),
          testComplexes: ("PPA", "Complex"),
          complexesGPEs: ("GPE", "Complex")}

shapeList = list(myDict.keys())

for key, value in myDict.items():   
    
    arcpy.AddField_management(key, "PPA_GPE", "TEXT")
    arcpy.AddField_management(key, "Park_Complex", "TEXT")
    
    with arcpy.da.UpdateCursor(key, ["PPA_GPE", "Park_Complex"]) as cursor:
        for row in cursor:
            row[0] = value[0]
            row[1] = value[1]
            cursor.updateRow(row)
        
PPA_GPEs = arcpy.Merge_management(shapeList, "PPA_GPEs")


parkFieldDict = {'ADMIN_AREA': 'ADMIN_AREA_SID', 
         'PROTECTED_': 'PROTECTED_LANDS_NAME', 
         'PROTECTED1': 'PROTECTED_LANDS_CODE', 
         'PROTECTE_1': 'PROTECTED_LANDS_DESIGNATION', 
         'ORCS_SECON': 'ORCS_SECONDARY', 
         'IUCN_Categ': 'IUCN_Category_Code', 
         'Park_Categ': 'Park_Category_Code', 
         'Establishe': 'Established_Date'}    

for key, value in parkFieldDict.items():
    arcpy.AlterField_management(PPA_GPEs, key, value, value)
    
    

arcpy.DeleteField_management(PPA_GPEs, ["Shape_Leng", "BUFF_DIST", "ORIG_FID"])     

# At this point, GPEs are generated. After this it is clipping the various variables to the PACEs. Theoretically, this could be a different notebook.


In [2]:
utm_7s = r"H:\utm\UTM_7S"
utm_8s = r"H:\utm\UTM_8S"
utm_9s = r"H:\utm\UTM_9S"
utm_10s = r"H:\utm\UTM_10S"
utm_11s = r"H:\utm\UTM_11S"

ntemsLocs = [utm_7s, utm_8s, utm_9s, utm_10s, utm_11s]
#print(ntemsLocs)

In [3]:
dist_7s = r"H:\utm\UTM_7S\Results\Change_attribution\Changes_attributed_logic_rules"
dist_8s = r"H:\utm\UTM_8s\Results\Change_attribution\Changes_attributed_logic_rules"
dist_9s = r"H:\utm\UTM_9s\Results\Change_attribution\Changes_attributed_logic_rules"
dist_10s = r"H:\utm\UTM_10s\Results\Change_attribution\Changes_attributed_logic_rules"
dist_11s = r"H:\utm\UTM_11s\Results\Change_attribution\Changes_attributed_logic_rules"

distLocs = [dist_7s, dist_8s, dist_9s, dist_10s, dist_11s]

In [4]:
for saveFolder, distFolder in zip(ntemsLocs, distLocs):
    
    shpList = []
    for file in os.listdir(distFolder):
        if file.endswith(".shp"):
            shpList.append(os.path.join(distFolder, file))
    
    distSaveLoc = os.path.join(saveFolder, "Change_attribution_logic_rules")
    if not os.path.isdir(distSaveLoc):
        os.makedirs(distSaveLoc)
        
    for shp in shpList:
        splitName = shp.rsplit('\\', 1)[-1].split(".")[0]
        saveName = os.path.join(distSaveLoc, splitName + ".tif")
        #print("running polygon to raster")
        if not os.path.isfile(saveName):
            field_names = [f.name for f in arcpy.ListFields(shp)]

            if "r_class" not in field_names:
                arcpy.AddField_management(shp, "r_class", "SHORT")

            with arcpy.da.UpdateCursor(shp, ["CLASS_LOGI", "r_class"]) as cursor:
                for row in cursor:
                    if row[0] == "Unclassified":
                        row[1] = 1000
                    if row[0] == "Fire":
                        row[1] = 1001
                    if row[0] == "Harvesting":
                        row[1] = 1002
                    if row[0] == "Lcondition":
                        row[1] = 1003
                    if row[0] == "Road":
                        row[1] = 1004
                    if row[0] == "Ag":
                        row[1] = 1005

                    cursor.updateRow(row)
        
        
            arcpy.PolygonToRaster_conversion(shp, "r_class", saveName, cellsize = 30)
        print("done " + saveName)
        
        
    print("done " + saveFolder)

done H:\utm\UTM_7S\Change_attribution_logic_rules\Logic_Rules_Change_Attribution_UTM_7S_1985.tif
done H:\utm\UTM_7S\Change_attribution_logic_rules\Logic_Rules_Change_Attribution_UTM_7S_1986.tif
done H:\utm\UTM_7S\Change_attribution_logic_rules\Logic_Rules_Change_Attribution_UTM_7S_1987.tif
done H:\utm\UTM_7S\Change_attribution_logic_rules\Logic_Rules_Change_Attribution_UTM_7S_1988.tif
done H:\utm\UTM_7S\Change_attribution_logic_rules\Logic_Rules_Change_Attribution_UTM_7S_1989.tif
done H:\utm\UTM_7S\Change_attribution_logic_rules\Logic_Rules_Change_Attribution_UTM_7S_1990.tif
done H:\utm\UTM_7S\Change_attribution_logic_rules\Logic_Rules_Change_Attribution_UTM_7S_1991.tif
done H:\utm\UTM_7S\Change_attribution_logic_rules\Logic_Rules_Change_Attribution_UTM_7S_1992.tif
done H:\utm\UTM_7S\Change_attribution_logic_rules\Logic_Rules_Change_Attribution_UTM_7S_1993.tif
done H:\utm\UTM_7S\Change_attribution_logic_rules\Logic_Rules_Change_Attribution_UTM_7S_1994.tif
done H:\utm\UTM_7S\Change_attr

done H:\utm\UTM_11S\Change_attribution_logic_rules\Logic_Rules_Change_Attribution_UTM_11S_1985.tif
done H:\utm\UTM_11S\Change_attribution_logic_rules\Logic_Rules_Change_Attribution_UTM_11S_1986.tif
done H:\utm\UTM_11S\Change_attribution_logic_rules\Logic_Rules_Change_Attribution_UTM_11S_1987.tif
done H:\utm\UTM_11S\Change_attribution_logic_rules\Logic_Rules_Change_Attribution_UTM_11S_1988.tif
done H:\utm\UTM_11S\Change_attribution_logic_rules\Logic_Rules_Change_Attribution_UTM_11S_1989.tif
done H:\utm\UTM_11S\Change_attribution_logic_rules\Logic_Rules_Change_Attribution_UTM_11S_1990.tif
done H:\utm\UTM_11S\Change_attribution_logic_rules\Logic_Rules_Change_Attribution_UTM_11S_1991.tif
done H:\utm\UTM_11S\Change_attribution_logic_rules\Logic_Rules_Change_Attribution_UTM_11S_1992.tif
done H:\utm\UTM_11S\Change_attribution_logic_rules\Logic_Rules_Change_Attribution_UTM_11S_1993.tif
done H:\utm\UTM_11S\Change_attribution_logic_rules\Logic_Rules_Change_Attribution_UTM_11S_1994.tif
done H:\ut

In [5]:
structVars = ["basal_area", "elev_cv", "gross_stem_volume",
             "loreys_height", "percentage_first_returns_above_2m", 
             "total_biomass"]

#unusedStructVars = ["elev_mean", "elev_p95", "elev_stddev", "percentage_first_returns_above_mean"]

input parameters: years, variables, ppas

In [6]:
shapefiles = []
for root, dirs, files in os.walk("outputs"):
    for file in files:
        if file.endswith("Subzones.shp"):
             shapefiles.append(os.path.join(root, file))
for_clipping = shapefiles
del shapefiles

In [3]:
for_clipping

['outputs\\AdamsLakeMarineParkPoplarPointSite\\AdamsLakeMarineParkPoplarPointSite_PACE_Subzones.shp',
 'outputs\\AdamsLakeMarineParkRefugeBaySite\\AdamsLakeMarineParkRefugeBaySite_PACE_Subzones.shp',
 'outputs\\AdamsLakeMarineParkSpillmanBeachSite\\AdamsLakeMarineParkSpillmanBeachSite_PACE_Subzones.shp',
 'outputs\\AdamsLakeParkBushCreekSite\\AdamsLakeParkBushCreekSite_PACE_Subzones.shp',
 'outputs\\AKK\\AKK_PACE_Subzones.shp',
 'outputs\\AlexandraBridgePark\\AlexandraBridgePark_PACE_Subzones.shp',
 'outputs\\AlezaLakeEcologicalReserve\\AlezaLakeEcologicalReserve_PACE_Subzones.shp',
 'outputs\\AliceLakePark\\AliceLakePark_PACE_Subzones.shp',
 'outputs\\AllisonHarbourMarinePark\\AllisonHarbourMarinePark_PACE_Subzones.shp',
 'outputs\\AllisonLakePark\\AllisonLakePark_PACE_Subzones.shp',
 'outputs\\AmbroseLakeEcologicalReserve\\AmbroseLakeEcologicalReserve_PACE_Subzones.shp',
 'outputs\\AnarchistProtectedArea\\AnarchistProtectedArea_PACE_Subzones.shp',
 'outputs\\AncientForestChunTOhWhudu

In [7]:
inVars = ["HMM", "disturbance", "disturbance_all", "fragstats", "elevation", "nightlights"] + structVars
years = [list(range(1984, 2020))]
ppaList = for_clipping

#inVars = ["disturbance", "HMM", "disturbance_all"]
#ppaList = ['C:\\Users\\evanmuis.stu\\Sync\\Masters\\Data\\outputs\\Gar\\Gar_PACE_Subzones.shp']
years = [2015]

print(inVars)
print(years)
print(ppaList)

['HMM', 'disturbance', 'disturbance_all', 'fragstats', 'elevation', 'nightlights', 'basal_area', 'elev_cv', 'gross_stem_volume', 'loreys_height', 'percentage_first_returns_above_2m', 'total_biomass']
[2015]
['outputs\\AdamsLakeMarineParkPoplarPointSite\\AdamsLakeMarineParkPoplarPointSite_PACE_Subzones.shp', 'outputs\\AdamsLakeMarineParkRefugeBaySite\\AdamsLakeMarineParkRefugeBaySite_PACE_Subzones.shp', 'outputs\\AdamsLakeMarineParkSpillmanBeachSite\\AdamsLakeMarineParkSpillmanBeachSite_PACE_Subzones.shp', 'outputs\\AdamsLakeParkBushCreekSite\\AdamsLakeParkBushCreekSite_PACE_Subzones.shp', 'outputs\\AKK\\AKK_PACE_Subzones.shp', 'outputs\\AlexandraBridgePark\\AlexandraBridgePark_PACE_Subzones.shp', 'outputs\\AlezaLakeEcologicalReserve\\AlezaLakeEcologicalReserve_PACE_Subzones.shp', 'outputs\\AliceLakePark\\AliceLakePark_PACE_Subzones.shp', 'outputs\\AllisonHarbourMarinePark\\AllisonHarbourMarinePark_PACE_Subzones.shp', 'outputs\\AllisonLakePark\\AllisonLakePark_PACE_Subzones.shp', 'outpu

In [14]:
def ntemsClipping(ppaList, inVars, years):
    
    numZones = 5
    
    x = 1
        
    df = pd.DataFrame(columns = ("park", "var", "year", "exists"))
    
    data = [i for i in it.product(ppaList, inVars, years)]
    num_files = len(data)
    num_done = 0
    
    for values in data:
        num_done += 1
        
        ppa = values[0]
        Var = values[1]
        year = values[2]
    
        ppaName = ppa.rsplit('\\', -1)[-1].split(".")[0].rsplit("_", 1)[0]
            
        VarSave = Var

        parkFolder = os.path.normpath(ppa + os.sep + os.pardir)
        saveFolder = os.path.join(parkFolder, "rasters")

        if not os.path.isdir(saveFolder):
            os.makedirs(os.path.join(saveFolder))
        
        print(str(num_done), "/", str(num_files))
        print(ppaName, Var, year)
        clear_output(wait = True)

        intSaveLocs = []

        finalSaveName = ppaName + "-" + str(year) + "-" + VarSave + ".tif"
        if Var == "elevation":
            finalSaveName = ppaName + "-elevation.tif"
        if Var == "disturbance_all":
            finalSaveName = ppaName + "-disturbance_all.tif"
        if not os.path.isfile(os.path.join(saveFolder, finalSaveName)):

            #for variables that have a single combined layer across bc_albers
            if Var == "fragstats":
                loc = r"H:\Muise\F_noF_other"
                varLoc = os.path.join(loc, "F-noF_BC_" + str(year) + ".tif")
                final = arcpy.sa.ExtractByMask(varLoc, ppa)
                final.save(os.path.join(saveFolder, finalSaveName))

            elif Var == "elevation":
                #elevation
                loc = r"H:\Muise\Elevation"
                varLoc = os.path.join(loc, "DEM_BC.tif")
                paceElev = arcpy.sa.ExtractByMask(varLoc, ppa)
                paceElev.save(os.path.join(saveFolder, finalSaveName))

                #slope
                finalSaveName = ppaName + "-slope.tif"

                paceSlope = Slope(paceElev)
                paceSlope.save(os.path.join(saveFolder, finalSaveName))

                #aspect
                finalSaveName = ppaName + "-aspect.tif"

                paceAspect = Aspect(paceElev)
                paceAspect.save(os.path.join(saveFolder, finalSaveName))

            elif Var == "nightlights":
                loc = r"H:\Muise\nightlights"
                varLoc = os.path.join(loc, "bc-albers_nightlights_" + str(year) + "_reclass.tif")

                final = arcpy.sa.ExtractByMask(varLoc, ppa)
                final.save(os.path.join(saveFolder, finalSaveName))


            #for variables that do not have a single combined layer across bc_albers  
            else:
                for i in range(numZones):

                    zoneNum = str(7 + i) + "S_"
                    zone = "UTM_" + str(7 + i) + "S"
                    zoneDistAll = "UTM" + str(7 + i) + "S"

                    #this has to be changed to work on other variables

                    if Var is "HMM":
                        loc = r"H:\VLCE"
                        varLoc = os.path.join(loc, zone, Var, "LC_Class_HMM_" + zoneNum + str(year) + ".dat")

                    elif Var in structVars:
                        loc = r"H:\Structure"
                        varLoc = os.path.join(loc, zone, Var, "UTM_" + zoneNum + Var + "_" + str(year) + ".dat")

                    elif Var is "disturbance":
                        loc = r"H:\utm"
                        varLoc = os.path.join(loc, zone, "Change_attribution_logic_rules", "Logic_Rules_Change_Attribution_UTM_" + zoneNum + str(year) + ".tif")

                    elif Var is "disturbance_all":
                        loc = r"H:\utm"
                        varLoc = os.path.join(loc, zone, "Results", "Change_attribution", "Attribution_"  + zoneDistAll + "_v2.dat")


                    if not os.path.isfile(os.path.join(saveFolder, finalSaveName)): 
                        #print(varLoc)
                        try:
                            interim = arcpy.sa.ExtractByMask(varLoc, ppa)
                            intSaveLoc = os.path.join(scratch, "test_" + zone + ".tif")
                            interim.save(intSaveLoc)
                            intSaveLocs.append(intSaveLoc)

                        except arcpy.ExecuteError:
                            continue




                #print(intSaveLocs)
                try:
                    if Var in structVars:
                        arcpy.CreateRasterDataset_management(scratch, finalSaveName, "30",
                             "16_BIT_UNSIGNED", number_of_bands = 1)

                        arcpy.Mosaic_management(intSaveLocs, os.path.join(scratch, finalSaveName))
                        final = arcpy.sa.ExtractByMask(os.path.join(scratch, finalSaveName), ppa)
                        forMaskLoc = r"H:\Muise\forestMask"
                        forMaskFile = "ForMask_UTM-7s-11S_" + str(year) + ".tif"
                        yearForMask = os.path.join(forMaskLoc, forMaskFile)
                        final = Times(final, yearForMask)
                        final.save(os.path.join(saveFolder, finalSaveName))

                    elif Var is "disturbance":
                        arcpy.CreateRasterDataset_management(scratch, finalSaveName, "30",
                                 "16_BIT_UNSIGNED", number_of_bands = 1)

                        interimMosaic = arcpy.Mosaic_management(intSaveLocs, os.path.join(scratch, finalSaveName))
                        interimMosaic = Con(IsNull(interimMosaic), 0, interimMosaic)
                        final = arcpy.sa.ExtractByMask(interimMosaic, ppa)

                        final.save(os.path.join(saveFolder, finalSaveName))

                    elif Var is "HMM":
                        arcpy.CreateRasterDataset_management(scratch, finalSaveName, "30",
                                 "16_BIT_UNSIGNED", number_of_bands = 1)

                        arcpy.Mosaic_management(intSaveLocs, os.path.join(scratch, finalSaveName))
                        final = arcpy.sa.ExtractByMask(os.path.join(scratch, finalSaveName), ppa)
                        final.save(os.path.join(saveFolder, finalSaveName))

                    else:
                        arcpy.CreateRasterDataset_management(scratch, finalSaveName, "30",
                                 "16_BIT_UNSIGNED", number_of_bands = 1)

                        arcpy.Mosaic_management(intSaveLocs, os.path.join(scratch, finalSaveName))
                        final = arcpy.sa.ExtractByMask(os.path.join(scratch, finalSaveName), ppa)
                        final.save(os.path.join(saveFolder, finalSaveName))

                    df.loc[x] = (ppaName, VarSave, year, "worked")
                except arcpy.ExecuteError:

                    df.loc[x] = (ppaName, VarSave, year, "did not work")
                    continue
        else:
            df.loc[x] = (ppaName, VarSave, year, "already exists")
        x += 1
                
    return df

In [12]:
outDf = ntemsClipping(ppaList, inVars, years)

10944 / 10944
ZumtelaBayConservancy_PACE total_biomass 2015


In [28]:
disturbance_dataframe = ntemsClipping(ppaList, ["disturbance"], list(range(1984, 2020)))

did not work


KeyboardInterrupt: 

In [15]:
disturbance_dataframe = ntemsClipping(ppaList, ["disturbance"], [1985])

912 / 912
ZumtelaBayConservancy_PACE disturbance 1985


In [16]:
disturbance_dataframe

,park,var,year,exists
1,AdamsLakeMarineParkPoplarPointSite_PACE,disturbance,1985,worked
2,AdamsLakeMarineParkRefugeBaySite_PACE,disturbance,1985,worked
3,AdamsLakeMarineParkSpillmanBeachSite_PACE,disturbance,1985,worked
4,AdamsLakeParkBushCreekSite_PACE,disturbance,1985,worked
5,AKK_PACE,disturbance,1985,worked
...,...,...,...,...
908,YalakomPark_PACE,disturbance,1985,worked
909,YaleGarryOakEcologicalReserve_PACE,disturbance,1985,worked
910,YardCreekPark_PACE,disturbance,1985,worked
911,YellowpointBogEcologicalReserve_PACE,disturbance,1985,worked


function to clip the ntems lcc and structure data, all inputs must be in list format

outDf = ntemsClipping(ppaList, inVars, years)

In [14]:
pd.set_option("display.max_rows", None)
display(outDf)

,park,var,year,exists
1,AdamsLakeMarineParkPoplarPointSite_PACE,HMM,2015,already exists
2,AdamsLakeMarineParkPoplarPointSite_PACE,disturbance,2015,already exists
3,AdamsLakeMarineParkPoplarPointSite_PACE,disturbance_all,2015,already exists
4,AdamsLakeMarineParkPoplarPointSite_PACE,fragstats,2015,already exists
5,AdamsLakeMarineParkPoplarPointSite_PACE,elevation,2015,already exists
6,AdamsLakeMarineParkPoplarPointSite_PACE,nightlights,2015,already exists
7,AdamsLakeMarineParkPoplarPointSite_PACE,basal_area,2015,already exists
8,AdamsLakeMarineParkPoplarPointSite_PACE,elev_cv,2015,already exists
9,AdamsLakeMarineParkPoplarPointSite_PACE,gross_stem_volume,2015,already exists
10,AdamsLakeMarineParkPoplarPointSite_PACE,loreys_height,2015,already exists


# Variables are now clipped to the PACE, next need to aggregate average data to subzones: New notebook start point?

In [15]:
structVars = ["basal_area", "elev_cv", "gross_stem_volume",
             "loreys_height", "percentage_first_returns_above_2m", 
             "total_biomass"]
meanVars = structVars + ["elevation", "slope", "aspect", "insolation"]

meanVars = tuple([s + ".tif" for s in meanVars])

In [25]:
def zStatsMean(fName, subzone):
    metadata = fName.split("\\")[-1].split(".")[0].split("-")
    var = metadata[-1]
    if var in structVars:
        year = metadata[1]
    else:
        year = None
    tSaveLoc = os.path.join(scratch, fName.split("\\")[-1].split(".")[0] + ".dbf")
    zonalOut = ZonalStatisticsAsTable(subzone, "myFID", fName, tSaveLoc, "DATA", "MEAN")
    arcpy.JoinField_management(zonalOut, "myFID", subzone, "myFID", ["ppa_gpe", "ZONE", "SUBZONE", "VARIANT", "PHASE", "NATURAL_DI", "Shape_Area"])
    if var in structVars:
        arcpy.CalculateField_management(zonalOut, "year", year, field_type = "SHORT")
    arcpy.CalculateField_management(zonalOut, "var", "var", field_type = "TEXT")
    arcpy.CalculateField_management(zonalOut, "park", "parkName", field_type = "TEXT")
    csv_save = fName.split("\\")[-1].split(".")[0] + ".csv"
    zStats_csv = arcpy.TableToTable_conversion(zonalOut, scratch, csv_save)
    return os.path.join(scratch, csv_save)

In [26]:
num_done = 0

In [27]:
d = 'outputs'
parkFolders = [os.path.join(wd, d, o) for o in os.listdir(d) 
                    if os.path.isdir(os.path.join(d,o))]

outDbfs = []

for park in parkFolders[num_done:]:
    files_done = 0
    parkName = park.rsplit("\\", 1)[-1]
    #print(parkName)
    subzone = os.path.join(park, parkName + "_PACE_Subzones.shp")
    #print(subzone)
    rasterLoc = os.path.join(park, "rasters")
    #print(rasterLoc)
    if os.path.isdir(rasterLoc):
        num_rasters = (len([name for name in os.listdir(rasterLoc) if name.endswith(meanVars)]))

        for file in os.listdir(rasterLoc):
            #for meanVar in meanVars:
            if file.endswith(meanVars):
                clear_output(True)
                print(str(num_done), "/", str(len(parkFolders)), "parks")
                print(files_done, "/", num_rasters, "working on", file)
                
                fileLoc = os.path.join(rasterLoc, file)
                metadata = file.split(".")[0].split("-")
                var = metadata[-1]
                outDbfs.append(zStatsMean(fileLoc, subzone))
                files_done += 1
    num_done += 1

            
            
clear_output(True)
print("merging")

        #if file.endswith(propVars):
            #propLocs.append(os.path.join(rasterLoc, file))
combined_csv = pd.concat( [ pd.read_csv(f) for f in outDbfs ] )
combined_csv.to_csv(os.path.join("outputCsvs", "meanVars.csv"))

for file in os.listdir(scratch):
    os.remove(os.path.join(scratch, file))
print("done!")

1 / 979 parks
2 / 9 working on AdamsLakeMarineParkRefugeBaySite_PACE-2015-gross_stem_volume.tif


KeyboardInterrupt: 

In [37]:
folders = [os.path.join(folder.rsplit("\\")[0], folder.rsplit("\\")[1], "rasters") for folder in for_clipping]

In [65]:
folders
years = list(range(1985, 2019))

df = pd.DataFrame(it.product(folders, years, inVars), columns=['folder', 'year', "variable"])

In [66]:
df["park"] = df.folder.str.rsplit("\\").str.get(1)

In [72]:
disturbance_df = df[df.variable == "disturbance"]
other_df = df[df.year == 2015]
other_df = df[df.variable != "disturbance"]

In [75]:
all_files = pd.concat([disturbance_df, other_df])

In [82]:
all_files.to_csv("files_for_checking.csv")